# Unit testing in Python with pytest

author: **Filip Noworolnik**  
email: fnoworolnik@gmail.com

## Unit testing

* **unit** - smallest testable part
* testing individual units

Example system:    
> cook <- waiter <- customer

[Example reference](https://stackoverflow.com/questions/3622455/what-is-the-purpose-of-mock-objects)

## Simple test example

```python
# functions.py
def add(a, b):
    return a+b
```

```python
# test_functions.py
from pytest_training import functions

def test_add():
    assert functions.add(1, 2) == 3
```

To run:
    `
    > pytest
    `

- keyword: **assert**
- auto-discovering tests

## pytest-coverage

Use it when you want to know if your tests cover all of the code.

```python
# functions.py
(...)

def t_or_f(x):
    if x < 5:
        return True
    elif x < 10:
        return False
    else:
        return None
```
***

```python
# functions2.py
def multiply(a, b):
    return a*b
```

<br>To run: `> pytest --cov:proj_name tests/`

To see in terminal which lines are not covered use `--cov-report term-missing`

It is also possible to skip fully covered modules in report. Just use: `--cov-report term:skip-covered`

You can use both together:  `> pytest --cov-report term-missing:skip-covered --cov=proj_name tests/`

Reports can be included in html, xml or annotated source code files instead of terminal:
```shell
pytest --cov-report html:cov_html
        --cov-report xml:cov.xml
        --cov-report annotate:cov_annotate
        --cov=proj_name tests/
```


>**Tip:** Don't omit your tests in coverage!  
If you have some helper code, not used anymore or odd part that you forget why is there - coverage would inform you about it.  
    It would also alert you if your tests have identical names (*Tests require names, but the names don't matter, because nothing calls the names directly. If there are two tests with identical names - you have only one test, because the second will overwrite the first one!*).  
    *Flake8*, *pylint* and some other linting tools can also alert you on some mistakes like same-named tests.

## Grouping test cases

### 1. Include multiple tests in class

```python
# test_functions.py
(...)

class TestTof:
    def test_tof_1(self):
        assert functions.t_or_f(4) is True

    def test_tof_2(self):
        assert functions.t_or_f(7) is False

    def test_tof_3(self):
        assert functions.t_or_f(15) is None
```

### 2. Use multiple asserts

```python
# test_functions.py
(...)

def test_tof():
    assert functions.t_or_f(4) is True
    assert functions.t_or_f(7) is False
    assert functions.t_or_f(15) is None
```

### 3. Parametrize test

```python
# test_functions.py
(...)

@pytest.mark.parametrize('test_input, expected', [(4, True), (7, False), (15, None)])
def test_tof_param(test_input, expected):
    assert functions.t_or_f(test_input) is expected
```

## Fixtures

Fixtures are objects that can be requested by the test functions or other fixtures by declaring them as argument names.

```python
@pytest.fixture
def db_session(tmpdir):
    fn = tmpdir / "db.file"
    return connect(str(fn))
```

## Temporary directories

`tmp_path` fixture provides a temporary directory unique to test invocation created in base temporary directory.

```python
CONTENT = "content"

def test_create_file(tmp_path):
    d = tmp_path / "sub"
    d.mkdir()
    p = d / "hello.txt"
    p.write_text(CONTENT)
    assert p.read_text() == CONTENT
    assert len(list(tmp_path.iterdir())) == 1
```

## skipping tests

If you want skip test (e.g. part of code is not yet implemented, so you don't want to test it), you can mark test to be skipped using `@skip` decorator.

```python
# functions.py
(...)

def not_implemented(a, b):
    ...
```
---

```python
# test_functions.py
(...)

@pytest.mark.skip(reason='Function is not implemented yet...')
def test_not_implemented():
    assert functions.not_implemented(5, 10) is True
```
  

The `reason` argument contains info about why the test is skipped and is included in report.
***

<br>  `skip()` can be also called as a function:

```python
# test_functions.py
(...)

def test_not_implemented2():
    pytest.skip('Function not implemented.')
```

Sometimes test have to be skipped under certain conditions, e.g. when test run on windows and function works only on linux.  
To achieve this, another function/decorator would be usefull: `skipif`
***

```python
# functions.py
(...)

def linux_function():
    print("This is linux-only function")
    if not sys.platform.startswith("win"):
        return True
    else:
        raise RuntimeError
```

```python
# test_functions.py
(...)

@pytest.mark.skipif(sys.platform.startswith("win"), reason="Test run on windows")
def test_linux_only():
    assert functions.linux_function() is True
```

## xfail, xpassed

Sometimes you expect the test to fail. You can mark it with `xfail`. The test will be run but no traceback will be reported when it fails.

```python
# functions.py
(...)

def impossible_function():
    print("This method runs only with certain configuration")
    response = requests.get('fifinow.com/amazing')
    return response.text
```

```python
# test_functions.py
(...)

@pytest.mark.xfail
def test_impossible():
    assert functions.impossible_function() == 'Filip is amazing!'
```

To run: `> pytest -rxXs`
***

If test unexpectedly pass, the report will mark it as `xpass`.

```python
# test_functions.py
(...)

def response_mock():
    return 'Filip is amazing!'


@pytest.mark.xfail
def test_impossible2(monkeypatch):
    monkeypatch.setattr(functions, 'impossible_function', response_mock)
    assert functions.impossible_function() == 'Filip is amazing!'
```
***

If you want to be more specific as to why the test is failing, you can specify exceptions in the `raises` argument.  
The test will be reported as a regular failure if it fails with an exception not mentioned in `raises`.

```python
# functions.py
(...)

def raises():
    x = list()
    return x[1]
```

```python
# test_functions.py
(...)

@pytest.mark.xfail(raises=IndexError)
def test_raises():
    assert functions.raises() == 5
```

## Mocking, monkey patching, and faking functionality

Let's test waiter:

> cook <- waiter <- test driver

1. test driver orders dishes and ensure the waiter returns correct dish
2. test depends on the cook:
    * cook may have non-deterministic behaviour (menu includes chef's surprise)
    * may have a lot of dependencies (won't cook without his entire staff)
    * may need a lot of resources (expensive ingredients, may take an hour to cook)
3. cook 'in cahoots' with test driver:
> test cook <- waiter <- test driver

Test cook can be implemented in different ways:
* fake cook - predending to be cook - using frozen dishes and microwave
* stub cook - hot dog vendor - always gives you hot dogs, no matter what order is
* mock cook - undercover cop, working with test driver

### 1. Monkey patching

Monkey patching allows you to intercept what a function would normally do. You can specify your own return value ans substitute it's full execution.

```python
# test_functions.py
(...)

def response_mock():
    return 'Filip is amazing!'

@pytest.fixture
def fix_response(monkeypatch):
    import functions
    monkeypatch.setattr(functions, 'impossible_function', response_mock)

def test_impossible_mpatched(fix_response):
    assert functions.impossible_function() == 'Filip is amazing!'
```

Here we created the substitute function using a `monkeypatch` object. Treating the `functions` module as an object, `monkeypatch` changes the behaviour of `impossible_function` inside the module when called to this test.

If you want to replace `impossible_function` behaviour across every test, you can use `autouse` keyword argument in `pytest.fixture`:

```python
# test_functions.py
(...)

def response_mock():
    return 'Filip is amazing!'

@pytest.fixture(autouse=True)
def fix_response(monkeypatch):
    import functions
    monkeypatch.setattr(functions, 'impossible_function', response_mock)

def test_impossible_mpatched():
    assert functions.impossible_function() == 'Filip is amazing!'
```

Despite of comfort of not having to include fixture as a parameter for every test, I would not recommend using it because of the risk of not being able to test the function after all.

### 2. MagicMock - faking functionality

Sometimes we need to patch over more than single function, like an instance of a whole object. This is a lot of work that we want to avoid.

```python
# functions.py
(...)

def getting_response(request):
    if request.method == 'GET':
        return {}
    if request.method == 'POST':
        return request.POST
```

```python
# test_functions.py
(...)

def test_getting_response_get(mocker):
    from pytest_training.functions import getting_response
    req = mocker.MagicMock()
    req.method = 'GET'
    assert getting_response(req) == {}
    
    
def test_getting_response_post(mocker):
    from pytest_training.functions import getting_response
    req = mocker.MagicMock()
    req.method = 'POST'
    req.POST = {'title': 'title', 'body': 'body'}
    assert getting_response(req) == {'title': 'title', 'body': 'body'}
```

We have created `MagicMock` object included in `pytest-mock` package with `method` attribute that has a value of `'GET'` in the first test and `'POST'` in the second. `POST` attribute is a dictionary containing some values.

`MagicMock` takes whatever form and functionality we need for the moment. It's great tool for unit testing that don't require checking for side effects. We may also mock side effects by providing `side_effects` keyword argument but when testing functions interconnecting with other parts of code we may want to choose a different testing method.

### 3. Tracking

If you don't need to completely hijack function, but keep track of it's usage, `mocker.spy` object from `pytes-mock` package is really usefull.

```python
# functions.py
(...)

class Numbers(object):
    def __init__(self, iterable):
        self._container = iterable

    def make_unique(self):
        i = 0
        visited = []
        while i < len(self._container):
            if self._container[i] in visited:
                self._drop_val(i)
                i = 0
                continue
            visited.append(self._container[i])
            i += 1

    def _drop_val(self, idx):
        self._container.pop(idx)
```

```python
# test_functions.py
(...)

def test_values_are_dropped_if_already_seen(mocker):
    nums = Numbers([1,2,1,2,1,2])
    mocker.spy(nums, '_drop_val')
    nums.make_unique()
    assert nums._make_unique.call_count == 4
```

## OMNI example

```python
import pytest

project_suids = {'TEE': 'HSBCKSHCXJSHSBFHFAS', 'CCU': 'DUFNCSHCXJSHSBIRUTJ'}


def return_suid():
    return project_suids


@pytest.fixture
def setup_creta(mocker, monkeypatch):
    mock_creta = mocker.MagicMock()
    mock_creta.get_project_suid.side_effect = return_suid
    return mock_creta


@pytest.fixture
def setup_msg_handler(mocker):
    msg_handler = mocker.MagicMock()
    return msg_handler


@pytest.fixture
def setup_logger(mocker):
    logger = mocker.MagicMock()
    return logger


@pytest.fixture
def setup_omni(setup_creta, setup_logger, tmpdir, setup_msg_handler):
    from omni.omni import Omni
    omni = Omni(setup_creta, setup_logger, tmpdir, setup_msg_handler)
    return omni


def test_run(setup_omni):
    assert setup_omni.run_omni() is True
```